# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [9]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [10]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [18]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head().sum(axis=1)

CustomerID
33     0.032348
200    0.039440
264    0.033545
356    0.034513
412    0.037069
dtype: float64

Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [20]:
#.div divide lo segundo entre lo primero, producto entre total productos cliente
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.000000,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.013298,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [22]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.853471,0.858742,0.853306,0.856845,0.851419,0.848198,0.847844,0.851494,0.847477,...,0.853547,0.846215,0.849391,0.840335,0.851613,0.846271,0.851671,0.846575,0.855182,0.850362
200,0.853471,1.000000,0.857755,0.856816,0.856528,0.852537,0.862793,0.850338,0.857746,0.856018,...,0.864431,0.856246,0.862707,0.853879,0.856899,0.854896,0.869280,0.858026,0.861805,0.856897
264,0.858742,0.857755,1.000000,0.852499,0.860711,0.850284,0.854831,0.846398,0.855225,0.847416,...,0.861643,0.850261,0.853136,0.848543,0.858594,0.860244,0.859258,0.853900,0.865122,0.852414
356,0.853306,0.856816,0.852499,1.000000,0.853916,0.850489,0.853008,0.851379,0.848804,0.845225,...,0.857006,0.848272,0.852503,0.847467,0.851569,0.851808,0.850509,0.854184,0.860359,0.854677
412,0.856845,0.856528,0.860711,0.853916,1.000000,0.854357,0.857896,0.857019,0.856104,0.854800,...,0.858121,0.862552,0.855099,0.849638,0.857780,0.856193,0.854690,0.855880,0.865306,0.861828


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [23]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,Pasta - Angel Hair,Tomatoes Tear Drop,Grouper - Fresh,Vaccum Bag 10x13
2,264,Scallops - 10/20,Wine - Ej Gallo Sierra Valley,Cheese - Camembert,Corn Meal,Browning Caramel Glace
3,356,Squid U5 - Thailand,Clam Nectar,"Turnip - White, Organic","Yogurt - Blueberry, 175 Gr",Garbag Bags - Black
4,412,Beef - Montreal Smoked Brisket,Scallops - 10/20,Onions - Cippolini,Juice - Orange,Cake - Box Window 10x10x2.5


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [24]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 33759, 6001, 79458]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [25]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [ ]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [32]:
prod_cust_pivot.head()
prod_dist=pd.DataFrame(1/(1+squareform(pdist(prod_cust_pivot,"euclidean"))), index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
prod_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477


### Step 2: Get the products purchased for a specific customer of your choice.

In [52]:
purchased_prod=list(prod_cust_pivot[27672][prod_cust_pivot[27672]!=0].index)
purchased_prod[:5]

['Assorted Desserts',
 'Baking Powder',
 'Bar Mix - Pina Colada, 355 Ml',
 'Barramundi',
 'Beef - Prime Rib Aaa']

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [59]:
top5sim={}
for product in purchased_prod:
    top5sim[product]=list(prod_dist[product].sort_values(ascending=False)[1:6])

In [65]:
not_in=[product for product in prod_cust_pivot[27672] if product not in top5sim.keys()]
not_in

AttributeError: 'float' object has no attribute 'index'

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [ ]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.